Implementing string matching for DNA sequences

In [1]:
from Levenshtein import distance
import numpy as np
from utils import create_random_strand

In [ ]:


def lev_distance(a, b):
    # Wagner fisher algorithm

    c = np.zeros((len(a), len(b)), dtype=int)

    for i in range(len(a)):
        c[i, 0] = i

    for i in range(len(b)):
        c[0, i] = i


    for i in range(1, len(a)):
        for j in range(1, len(b)):

            subsitution_cost = 0 if a[i] == b[j] else 1 # torch.where

            c[i, j] = min(
                c[i-1, j-1] + subsitution_cost,
                c[i-1, j] + 1,
                c[i, j-1] + 1
            ) # also can be a batch operation

    return c[i, j]



In [ ]:


def lev_distance_vec(a, b):
    # Wagner fisher algorithm
    # Might need same length for it to work - could just chop out the last few bases if needbe

    c = np.zeros((len(a), len(b)), dtype=int)

    for i in range(len(a)):
        c[i, 0] = i

    for i in range(len(b)):
        c[0, i] = i


    for i in range(1, len(a)):
        for j in range(1, len(b)):

            subsitution_cost = 0 if a[i] == b[j] else 1 # torch.where

            c[i, j] = min(
                c[i-1, j-1] + subsitution_cost,
                c[i-1, j] + 1,
                c[i, j-1] + 1
            ) # also can be a batch operation

    return c[i, j]



In [46]:
def sequence_alignment(a, b, match_score=1, mismatch_score=-1, gap_score=-1):
    """Traceback - 1 - diagonal, 2 - left, 3 - up"""

    score = np.zeros((len(a) + 1, len(b) + 1), dtype=int)
    traceback = np.zeros((len(a), len(b)), dtype=int)

    for i in range(len(a)):
        score[i, 0] = i * gap_score
        traceback[i, 0] = 3 if i > 0 else 0

    for i in range(len(b)):
        score[0, i] = (i * gap_score)
        traceback[0, i] = 2 if i > 0 else 0

    for i in range(1, len(a)):
        for j in range(1, len(b)):
            
            print(i, j)
            print(a[i], b[j])
            subsitution_cost = match_score if a[i] == b[j] else mismatch_score
            print(subsitution_cost)

            comparision_list = [
                score[i-1, j-1] + subsitution_cost,
                score[i - 1, j] + gap_score,
                score[i, j - 1] + gap_score
            ]
            print(comparision_list)
            print()

            cell_score = max(comparision_list)

            score[i, j] = cell_score
            traceback[i, j] = comparision_list.index(cell_score) + 1

    return score, traceback


In [43]:
t = "GCATGCG"
s = "GATTACA"

In [44]:
dis = lev_distance(t, s)

In [45]:
score, traceback = sequence_alignment(t, s)

1 1
C A
-1
[np.int64(-1), np.int64(-2), np.int64(-2)]

1 2
C T
-1
[np.int64(-2), np.int64(-3), np.int64(-2)]

1 3
C T
-1
[np.int64(-3), np.int64(-4), np.int64(-3)]

1 4
C A
-1
[np.int64(-4), np.int64(-5), np.int64(-4)]

1 5
C C
1
[np.int64(-3), np.int64(-6), np.int64(-5)]

1 6
C A
-1
[np.int64(-6), np.int64(-7), np.int64(-4)]

2 1
A A
1
[np.int64(0), np.int64(-2), np.int64(-3)]

2 2
A T
-1
[np.int64(-2), np.int64(-3), np.int64(-1)]

2 3
A T
-1
[np.int64(-3), np.int64(-4), np.int64(-2)]

2 4
A A
1
[np.int64(-2), np.int64(-5), np.int64(-3)]

2 5
A C
-1
[np.int64(-5), np.int64(-4), np.int64(-3)]

2 6
A A
1
[np.int64(-2), np.int64(-5), np.int64(-4)]

3 1
T A
-1
[np.int64(-3), np.int64(-1), np.int64(-4)]

3 2
T T
1
[np.int64(1), np.int64(-2), np.int64(-2)]

3 3
T T
1
[np.int64(0), np.int64(-3), np.int64(0)]

3 4
T A
-1
[np.int64(-3), np.int64(-3), np.int64(-1)]

3 5
T C
-1
[np.int64(-3), np.int64(-4), np.int64(-2)]

3 6
T A
-1
[np.int64(-4), np.int64(-3), np.int64(-3)]

4 1
G A
-1
[np.int64

In [28]:
score

array([[ 0, -1, -2, -3, -4, -5, -6],
       [-1, -1, -2, -3, -4, -3, -4],
       [-2,  0, -1, -2, -2, -3, -2],
       [-3, -1,  1,  0, -1, -2, -3],
       [-4, -2,  0,  0, -1, -2, -3],
       [-5, -3, -1, -1, -1,  0, -1],
       [-6, -4, -2, -2, -2, -1, -1]])

In [29]:
traceback

array([[0, 2, 2, 2, 2, 2, 2],
       [3, 1, 1, 1, 1, 1, 3],
       [3, 1, 3, 3, 1, 3, 1],
       [3, 2, 1, 1, 3, 3, 2],
       [3, 2, 2, 1, 1, 1, 1],
       [3, 2, 2, 1, 1, 1, 3],
       [3, 2, 2, 1, 1, 2, 1]])

In [33]:
base_lookup = {
    "A": 1,
    "C": 2,
    "G": 3,
    "T": 4
}

In [36]:
t_num = [base_lookup[i] for i in t]

In [40]:
s_num = np.array([base_lookup[i] for i in s])

In [38]:
t_num = np.array(t_num)

In [39]:
t_num

array([2, 1, 3, 1, 2, 3, 4, 4, 1, 4, 4, 2, 2, 3, 1, 4, 1, 2, 1, 2, 2, 4,
       3, 4, 4, 3, 1, 3, 3, 3, 4, 1, 1, 4, 3, 1, 3, 2, 3, 3, 2, 1, 3, 3,
       1, 1, 3, 2, 1, 3, 2, 3, 3, 1, 4, 2, 2, 3, 3, 1, 2, 2, 2, 3, 2, 4,
       1, 4, 1, 4, 2, 3, 2, 2, 4, 1, 2, 4, 4, 3, 1, 4, 1, 4, 2, 3, 3, 4,
       4, 4, 1, 4, 3, 3, 3, 1, 2, 3, 2, 3, 2, 3, 2, 2, 4, 2, 1, 2, 3, 1,
       4, 3, 4, 2, 2, 2, 1, 3, 3, 2, 4, 4, 2, 4, 3, 1, 4, 2, 3, 4])

In [41]:
s_num

array([2, 4, 2, 2, 1, 3, 4, 1, 4, 4, 2, 4, 2, 2, 1, 3, 3, 1, 3, 1, 4, 1,
       1, 1, 4, 4, 1, 1, 2, 3, 1, 4, 1, 1, 2, 4, 2, 4, 1, 1, 3, 1, 4, 3,
       2, 2, 3, 1, 1, 4, 2, 2, 3, 2, 4, 1, 1, 1, 4, 1, 2, 2, 1, 3, 3, 4,
       1, 1, 3, 3, 1, 3, 3, 3, 4, 2, 2, 2, 4, 3, 3, 4, 2, 2, 4, 1, 1, 4,
       1, 3, 2, 4, 1, 4, 1, 3, 2, 2, 3, 1, 1, 1, 1, 3, 2, 4, 1, 2, 2, 2,
       4, 1, 4, 3, 1, 1, 2, 1, 3, 2, 2, 1, 4, 1, 2, 4, 2, 2, 4, 2, 3, 4,
       4, 2, 3, 2, 2, 4, 3, 3, 1, 3, 3, 4, 1, 4, 3, 4, 4, 2, 4, 4, 4, 1,
       3, 2, 2, 1, 1, 4, 2, 1, 1, 3, 2, 1, 3, 2, 1, 3, 3, 4, 3, 3, 4, 3,
       1, 1, 3, 3, 4, 3, 1, 4, 2, 3, 4, 3, 3, 4, 1, 1, 4, 1, 4, 1, 2, 1,
       3, 4, 4, 4, 3, 3, 1, 4, 1, 4, 1, 2, 2, 3, 3, 1, 3, 4, 1, 3, 4, 2,
       3, 1, 2, 2, 1, 4, 3, 1, 3, 4])

In [42]:
i = 2

In [43]:
np.where(i == t_num, 0, 1)

array([0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1])